In [1]:
#create all rules
def exeva(i,a,s,num):
    if i == num:
        return a, s
    
    tempa = []
    temps = []
    
    temp = exeva(i+1,a+[1]+[0]+[0],s+'1',num)
    if i == num-1:
        tempa.append(temp[0])
        temps.append(temp[1])
    else:
        tempa.extend(temp[0])
        temps.extend(temp[1])
    
    temp = exeva(i+1,a+[0]+[1]+[0],s+'2',num)
    if i == num-1:
        tempa.append(temp[0])
        temps.append(temp[1])
    else:
        tempa.extend(temp[0])
        temps.extend(temp[1])
    
    temp = exeva(i+1,a+[0]+[0]+[1],s+'3',num)
    if i == num-1:
        tempa.append(temp[0])
        temps.append(temp[1])
    else:
        tempa.extend(temp[0])
        temps.extend(temp[1])
    
    return tempa, temps

In [2]:
def readdata(filename, dimension):
    #read data
    xdata = []
    with open(filename) as f:
        lines = f.readlines()
    tp = 0
    tdata = []
    for line in lines:
        tdata.append(float(line))
        tp = tp + 1
        if tp==dimension:
            tp = 0
            xdata.append(tdata)
            tdata = []
    return xdata

In [3]:
def initialize():
    #placeholder
    x = tf.placeholder(tf.float32, [None, D_in])
    y = tf.placeholder(tf.float32, [None, D_out])
    w = {
        'h1': tf.Variable(tf.random_normal([D_in, D_h1])),
        'h2': tf.Variable(tf.random_normal([D_h1, D_h2])),
        'h3': tf.Variable(tf.random_normal([D_h2, D_h3])),
        'h4': tf.Variable(tf.random_normal([D_h3, D_h4])),
        #'h5': tf.Variable(tf.random_normal([D_h4, D_h5])),
        'out': tf.Variable(tf.random_normal([D_h4, D_out]))
    }
    b = {
        'h1': tf.Variable(tf.random_normal([D_h1])),
        'h2': tf.Variable(tf.random_normal([D_h2])),
        'h3': tf.Variable(tf.random_normal([D_h3])),
        'h4': tf.Variable(tf.random_normal([D_h4])),
        #'h5': tf.Variable(tf.random_normal([D_h5])),
        'out': tf.Variable(tf.random_normal([D_out]))
    }

    #activation functions #tf.layers.batch_normalization()
    def multilayer_perceptron(x):
        h1_layer = tf.sigmoid(tf.add(tf.matmul(x, w['h1']), b['h1']))
        h2_layer = tf.sigmoid(tf.add(tf.matmul(h1_layer, w['h2']), b['h2']))
        h3_layer = tf.sigmoid(tf.add(tf.matmul(h2_layer, w['h3']), b['h3']))
        h4_layer = tf.sigmoid(tf.add(tf.matmul(h3_layer, w['h4']), b['h4']))
        #h5_layer = tf.sigmoid(tf.add(tf.matmul(h4_layer, w['h5']), b['h5']))
        out_layer = tf.sigmoid(tf.add(tf.matmul(h4_layer, w['out']), b['out']))
        return out_layer
    pred = multilayer_perceptron(x)

    #loss function
    #cost = tf.reduce_sum(tf.where(tf.equal(0.0, y), abs(y-pred),
    #                     tf.where(tf.equal(1.0, y), abs(y-pred),
    #                     tf.where(tf.equal(-1.0, y), y-y,
    #                     (y-pred)*(y-pred)))))
    cost = tf.reduce_sum((y-pred)*(y-pred))

    #optimizer and others
    optimizer = tf.train.AdamOptimizer(0.002).minimize(cost)
    init = tf.global_variables_initializer()
    variables_dict = {
        'b1': b['h1'],
        'b2': b['h2'],
        'b3': b['h3'],
        'b4': b['h4'],
        #'b5': b['h5'],
        'bout': b['out'],
        'w1': w['h1'],
        'w2': w['h2'],
        'w3': w['h3'],
        'w4': w['h4'],
        #'w5': w['h5'],
        'wout': w['out']
    }
    saver = tf.train.Saver(variables_dict)
    
    tf.summary.scalar('loss',cost)
    tf.summary.histogram('b1', b['h1'])
    tf.summary.histogram('b2', b['h2'])
    tf.summary.histogram('b3', b['h3'])
    tf.summary.histogram('b4', b['h4'])
    tf.summary.histogram('bout', b['out'])
    tf.summary.histogram('w1', w['h1'])
    tf.summary.histogram('w2', w['h2'])
    tf.summary.histogram('w3', w['h3'])
    tf.summary.histogram('w4', w['h4'])
    tf.summary.histogram('wout', w['out'])
    
    return x, y, pred, cost, optimizer, init, saver

In [4]:
def evaluate(write):
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        saver.restore(sess, "./model.ckpt")
        result = pred.eval(feed_dict = {x: edata})
    if write:
        with open('nnr.txt', 'w') as f:
            for result1 in result:
                for result2 in result1:
                    f.write(str(result2) + '\n')
    return result

In [5]:
def train():
    with tf.Session() as sess:
        sess.run(init)
        merged_all = tf.summary.merge_all()
        writer = tf.summary.FileWriter('logs/', sess.graph)
        for epoch in range(training_epochs):
            avg_cost = 0.
            total_batch = int(num/batch_size)
            for i in range(total_batch):
                batch_xs = xdata[i*batch_size:(i+1)*batch_size]
                batch_ys = ydata[i*batch_size:(i+1)*batch_size]
                _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs, y: batch_ys})
                avg_cost += c / total_batch
            if (epoch+1) % display_step == 0 or epoch == 0:
                print("Training Loop "+str(loopcount), "Epoch:", '%04d' % (epoch+1), " cost=", "{:.9f}".format(avg_cost))
                #merged = sess.run(merged_all, feed_dict={x: batch_xs, y: batch_ys})
                #writer.add_summary(merged, epoch)
            if (epoch+1) % save_step == 0:
                saver.save(sess, "./model.ckpt")
                print("save model done.")

In [6]:
def train_CV(bearcount):
    minscore = 1.0
    count = 0
    with tf.Session() as sess:
        sess.run(init)
        merged_all = tf.summary.merge_all()
        writer = tf.summary.FileWriter('logs/', sess.graph)
        for epoch in range(training_epochs):
            avg_cost = 0.
            total_batch = int(num/batch_size)
            for i in range(total_batch):
                batch_xs = xdata[i*batch_size:(i+1)*batch_size]
                batch_ys = ydata[i*batch_size:(i+1)*batch_size]
                _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs, y: batch_ys})
                avg_cost += c / total_batch
                
            evadata = pred.eval(feed_dict = {x: edata})
            score = 0
            count = count + 1
            for j in range(len(evadata)):
                score = score + abs(evadata[j]-rdata[j])/len(evadata)
            if score<minscore:
                minscore = score
                count = 0
            if count>bearcount:
                saver.save(sess, "./model.ckpt")
                print("save model done.")
                print("Training Loop "+str(loopcount), "Epoch:", '%04d' % (epoch+1), " cost=", "{:.9f}".format(avg_cost))
                print('Training Loop '+str(loopcount)+' Epoch: '+str((epoch+1))+' no further improvement in the testing set, stop.')
                print('current score '+str(score)+' minscore '+str(minscore))
                break
            
            if (epoch+1) % display_step == 0 or epoch == 0:
                print("Training Loop "+str(loopcount), "Epoch:", '%04d' % (epoch+1), " cost=", "{:.9f}".format(avg_cost))
                #merged = sess.run(merged_all, feed_dict={x: batch_xs, y: batch_ys})
                #writer.add_summary(merged, epoch)
            if (epoch+1) % save_step == 0:
                saver.save(sess, "./model.ckpt")
                print("save model done.")

In [7]:
def find():
    #generate all rules
    exdata, exdatastr = exeva(0,[],'2',13)
    
    #predict output
    with tf.Session() as sess:
        saver.restore(sess, "./model.ckpt")
        result = pred.eval(feed_dict = {x: exdata})

    #sort results
    sortedresult = []
    for i in range(len(result)):
        sortedresult.append((exdatastr[i],result[i]))
    dtype = [('Rule', 'S14'),('Value', float)]
    sortedresult = np.array(sortedresult, dtype=dtype)
    sortedresult = np.sort(sortedresult, order='Value')
    
    #write new rules, top 100 minimum, top 100 maximum, equally-separated 100
    with open('newrules.txt', 'w') as f:
        for i in range(len(sortedresult)):
            if (i<100) or (i>len(sortedresult)-1-100) or (i%((len(sortedresult)-200)//100)==0):
                f.write(str(sortedresult[i][0])[2:-1] + '\n')

    #write new volumes
    with open('newvolumes.txt', 'w') as f:
        for i in range(len(sortedresult)):
            if (i<100) or (i>len(sortedresult)-1-100) or (i%((len(sortedresult)-200)//100)==0):
                f.write(str(sortedresult[i][1]) + '\n')
                
    print('find doen!')

In [8]:
def clean():
    xx = []
    yy = []
    pp = [True for i in range(len(xdata))]
    for i in range(len(xdata)-1):
        for j in range(i+1,len(xdata)):
            pd = True
            for k in range(D_in):
                if xdata[i][k] != xdata[j][k]:
                    pd = False
                    break
            if pd == True:
                pp[i] = False
                break
    for i in range(len(xdata)):
        if pp[i] == True:
            xx.append(xdata[i])
            yy.append(ydata[i])
    print('clean doen! '+str(len(xx))+' instances from '+str(len(xdata))+' instances left.')
    return xx, yy

In [9]:
def shuffledata():
    
    import random
    
    xdatatotal = xdata
    ydatatotal = ydata
    numtotal = len(xdatatotal)
    datatotal = []
    for i in range(numtotal):
        xdatatotal[i].extend(ydatatotal[i])
        datatotal.append(xdatatotal[i])
    random.shuffle(datatotal)
    
    xx = [[] for i in range(len(datatotal))]
    yy = [[] for i in range(len(datatotal))]
    for i in range(len(datatotal)):
        for j in range(D_in):
            xx[i].append(datatotal[i][j])
        for j in range(D_in,D_in+D_out):
            yy[i].append(datatotal[i][j])
    return xx, yy

In [10]:
def reassign(fold):
    xx = []
    yy = []
    ee = []
    rr = []
    for j in range(len(xdata)):
        if j<=int(len(xdata)/fold):
            ee.append(xdata[j])
            rr.append(ydata[j])
        else:
            xx.append(xdata[j])
            yy.append(ydata[j])
    return xx, yy, ee, rr

In [12]:
#import
import tensorflow as tf
import numpy as np
import random

#training settings
training_epochs = 20000
batch_size = 50
display_step = 100
save_step = 5000

#network structure
D_in, D_out = 39, 1
D_h1 = 200
D_h2 = 100
D_h3 = 50
D_h4 = 25
#D_h5 = 12

#initialize
x, y, pred, cost, optimizer, init, saver = initialize()

action = 'find'
action = '5_fold_CV'

if action == 'train':
    #load data
    xdata = readdata('nnin.txt', D_in)
    ydata = readdata('nnout.txt', D_out)
    num = len(xdata)

    #train
    loopcount = 1
    train()

elif action == 'train_CV':
    #load data
    xdata = readdata('nnin.txt', D_in)
    ydata = readdata('nnout.txt', D_out)
    num = len(xdata)
    
    #clean and shuffle and reassign data
    xdata, ydata = clean()
    xdata, ydata = shuffledata()
    xdata, ydata, edata, rdata = reassign(5)
    num = len(xdata)
    
    loopcount = 0
    train_CV(200)
    
    
elif action == 'find':
    #finding the 300 pieces of data
    find()
    
elif action == 'evaluate':
    edata = readdata('nne.txt', D_in)
    evaluate(True)
    
elif action == '5_fold_CV':
    xdatatotal = readdata('nnin.txt', D_in)
    ydatatotal = readdata('nnout.txt', D_out)
    numtotal = len(xdatatotal)
    datatotal = []
    for i in range(numtotal):
        xdatatotal[i].extend(ydatatotal[i])
        datatotal.append(xdatatotal[i])
    random.shuffle(datatotal)
    
    finalscore = 0
    for i in range(1):
        xdata = []
        ydata = []
        edata = []
        rdata = []
        for j in range(numtotal):
            if j<=int(numtotal/5*(i+1)) and j>=int(numtotal/5*i):
                edata.append([datatotal[j][k] for k in range(D_in)])
                rdata.append([datatotal[j][D_in]])
            else:
                xdata.append([datatotal[j][k] for k in range(D_in)])
                ydata.append([datatotal[j][D_in]])
        num = len(xdata)
        
        loopcount = i
        train()
        evadata = evaluate(False)
        score = 0
        for j in range(len(evadata)):
            score = score + abs(evadata[j]-rdata[j])/len(evadata)
        print('5_flod_CV step accuracy:', score)
        finalscore = finalscore + score/5
    print('5_flod_CV average accuracy:', finalscore)

Training Loop 0 Epoch: 0001  cost= 0.653845139
Training Loop 0 Epoch: 0100  cost= 0.000340266
Training Loop 0 Epoch: 0200  cost= 0.000263941
Training Loop 0 Epoch: 0300  cost= 0.000991438
Training Loop 0 Epoch: 0400  cost= 0.000398808
Training Loop 0 Epoch: 0500  cost= 0.000316970
Training Loop 0 Epoch: 0600  cost= 0.002351369
Training Loop 0 Epoch: 0700  cost= 0.000185682
Training Loop 0 Epoch: 0800  cost= 0.000260276
Training Loop 0 Epoch: 0900  cost= 0.000432813
Training Loop 0 Epoch: 1000  cost= 0.001182136
Training Loop 0 Epoch: 1100  cost= 0.004502316
Training Loop 0 Epoch: 1200  cost= 0.004144495
Training Loop 0 Epoch: 1300  cost= 0.002929034
Training Loop 0 Epoch: 1400  cost= 0.003489850
Training Loop 0 Epoch: 1500  cost= 0.007252704
Training Loop 0 Epoch: 1600  cost= 0.001719130
Training Loop 0 Epoch: 1700  cost= 0.006686851
Training Loop 0 Epoch: 1800  cost= 0.002781973
Training Loop 0 Epoch: 1900  cost= 0.004034559
Training Loop 0 Epoch: 2000  cost= 0.003368126
Training Loop

Training Loop 0 Epoch: 17200  cost= 0.002848237
Training Loop 0 Epoch: 17300  cost= 0.002157818
Training Loop 0 Epoch: 17400  cost= 0.002357084
Training Loop 0 Epoch: 17500  cost= 0.002449370
Training Loop 0 Epoch: 17600  cost= 0.002721618
Training Loop 0 Epoch: 17700  cost= 0.002279713
Training Loop 0 Epoch: 17800  cost= 0.001479817
Training Loop 0 Epoch: 17900  cost= 0.002085682
Training Loop 0 Epoch: 18000  cost= 0.001711413
Training Loop 0 Epoch: 18100  cost= 0.001745965
Training Loop 0 Epoch: 18200  cost= 0.002646277
Training Loop 0 Epoch: 18300  cost= 0.002167450
Training Loop 0 Epoch: 18400  cost= 0.002740120
Training Loop 0 Epoch: 18500  cost= 0.002571063
Training Loop 0 Epoch: 18600  cost= 0.002935673
Training Loop 0 Epoch: 18700  cost= 0.003119225
Training Loop 0 Epoch: 18800  cost= 0.003052456
Training Loop 0 Epoch: 18900  cost= 0.002541199
Training Loop 0 Epoch: 19000  cost= 0.002218225
Training Loop 0 Epoch: 19100  cost= 0.001531305
Training Loop 0 Epoch: 19200  cost= 0.00

Training Loop 1 Epoch: 14100  cost= 0.002435571
Training Loop 1 Epoch: 14200  cost= 0.001093224
Training Loop 1 Epoch: 14300  cost= 0.002067856
Training Loop 1 Epoch: 14400  cost= 0.001883319
Training Loop 1 Epoch: 14500  cost= 0.002149766
Training Loop 1 Epoch: 14600  cost= 0.002170822
Training Loop 1 Epoch: 14700  cost= 0.002696654
Training Loop 1 Epoch: 14800  cost= 0.002053957
Training Loop 1 Epoch: 14900  cost= 0.001511528
Training Loop 1 Epoch: 15000  cost= 0.001648417
save model done.
Training Loop 1 Epoch: 15100  cost= 0.001102393
Training Loop 1 Epoch: 15200  cost= 0.001241891
Training Loop 1 Epoch: 15300  cost= 0.001109855
Training Loop 1 Epoch: 15400  cost= 0.001675562
Training Loop 1 Epoch: 15500  cost= 0.002082233
Training Loop 1 Epoch: 15600  cost= 0.002011656
Training Loop 1 Epoch: 15700  cost= 0.002173467
Training Loop 1 Epoch: 15800  cost= 0.002365625
Training Loop 1 Epoch: 15900  cost= 0.003301967
Training Loop 1 Epoch: 16000  cost= 0.003070128
Training Loop 1 Epoch: 

KeyboardInterrupt: 

In [21]:
xdatatotal = np.asarray(readdata('nnin.txt', D_in))
ydatatotal = np.asarray(readdata('nnout.txt', D_out))

In [22]:
xdatatotal.shape

(1281, 39)

In [23]:
ydatatotal.shape

(1281, 1)